This notebook will take a dataset of images, run them through TSNE to group them up (if enabled) then create a stylegan2 model with or without ADA.

Below are setting to choose when running this workflow. Make sure before running to have all images you want to use in a folder inside of the images folder. For example have a folder inside images called mona-lisa filled with pictures of different versions of the Mona Lisa. Please have the subfolder have no whitespaces in the name.

If TSNE is enable the program will halt after processing the images and ask you to choose which cluster to use. The clusters will be in the folder clusters.

Before running make sure your kernal is set to Python 3 (TensorFlow 1.15 Python 3.7 GPU Optimized)

In [40]:
dataset_name = 'mona-lisa'

use_ada = True
use_tsne = True
use_spacewalk = True


# Crop Settings
# Choose center, or resize
crop_type = 'resize'
resolution = 512

# TSNE Settings
# Choose number of clusters to make or None for auto clustering
num_clusters = None

# ADA Settings
knum = 10

# Spacewalk Settings
fps = 24
seconds = 10
#Leave seeds = None for random seeds or 
# enter a list in the form of [int, int, int..] to define the seeds
seeds = None
# set walk_type to 'line', 'sphere', 'noiseloop', or 'circularloop'
walk_type = 'sphere'


In [6]:
!pip install -r requirements.txt

You should consider upgrading via the '/usr/local/bin/python3.7 -m pip install --upgrade pip' command.


In [12]:
import os
import train
from PIL import Image
import shutil

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [17]:
def resize(pil_img, res):
    return pil_img.resize((res, res))

def crop_center(pil_img, res):
    
    crop = res
    
    img_width, img_height = pil_img.size
    if img_width < crop:
        crop = img_width
    if img_height < crop:
        crop = img_height
        
    a = (img_width - crop) // 2
    b = (img_height - crop) // 2
    c = (img_width + crop) // 2
    d = (img_height + crop) // 2
        
    cropped_image = pil_img.crop((a,b,c,d))
    return resize(cropped_image, res)

In [23]:
image_dir = './images/'
tmp_dir = './tmp/'

image_dir = os.path.join(image_dir, dataset_name)
tmp_dir = os.path.join(tmp_dir, dataset_name)


if not os.path.exists(save_dir):
    os.makedirs(save_dir)
else:
    try:
        shutil.rmtree(save_dir)
    except OSError as e:
        print("Error: %s : %s" % (dir_path, e.strerror))
    os.makedirs(save_dir)
    

for filename in os.listdir(image_dir):
    file_extension = os.path.splitext(filename)[-1]
    if file_extension != '.jpg' and file_extension != '.png':
        print(file_extension)
        continue
    
    image_path = os.path.join(image_dir, filename)
    image = Image.open(image_path)
    mode = image.mode
    if str(mode) != 'RGB':
        continue
    if crop_type == "center":
        image = crop_center(image, resolution)
    if crop_type == "resize":
        image = resize(image, resolution)
        
    tmp_path = os.path.join(tmp_dir, filename)
    image.save(tmp_path)

In [41]:
if use_tsne:
    !python tsne.py --path={tmp_dir}